Install Databricks SDK

In [0]:
%pip install databricks-sdk --upgrade

  Obtaining dependency information for databricks-sdk from https://files.pythonhosted.org/packages/ab/4e/78dc7ecbd7ffe828a2657e88e8e5f2a4bac0942f7a7d6a91f49d5b72aa44/databricks_sdk-0.36.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/569.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: databricks-sdk
    Found existing installation: databricks-sdk 0.20.0
    Not uninstalling databricks-sdk at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-37d204a1-bc55-48cc-8b4d-39435304f884
    Can't uninstall 'databricks-sdk'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip show databricks-sdk | grep -oP '(?<=Version: )\S+'

0.36.0


Create Databricks job

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import Task, PipelineTask, Source

w = WorkspaceClient()

job_name = "Databricks SDK Job"
description = "Databricks SDK Job to Run a Delta Live Tables Data Pipeline"
pipeline_id = "ce45eeba-9337-4580-9b4a-402ac3ae231f"
task_key = "dlt_pipeline_task"

print("Attempting to create the job. Please wait...\n")

j = w.jobs.create(
    name = job_name,
    tasks = [
        Task(
            description = description,           
            pipeline_task = PipelineTask(
                pipeline_id=pipeline_id,
                full_refresh=False                
            ),
            task_key = task_key
        )
    ]
)

print(f"View the job at {w.config.host}/#job/{j.job_id}\n")

Attempting to create the job. Please wait...

View the job at https://westus.azuredatabricks.net/#job/167559915304449



In [0]:
run_now_response = w.jobs.run_now(job_id=j.job_id)
run_now_response.response.run_id

620884104255456

Created new Delta Live Tables pipeline update with ID: 841f563c-7fba-460b-939b-282ee46ca592 from existing one, ce45eeba-9337-4580-9b4a-402ac3ae231f

Cancel job

In [0]:
cancelled_run = w.jobs.cancel_run(run_id=run_now_response.response.run_id).result()
cancelled_run

Run(attempt_number=None, cleanup_duration=0, cluster_instance=None, cluster_spec=None, creator_user_name='odl_user_1503114@databrickslabs.com', end_time=1731015216612, execution_duration=73000, git_source=None, job_clusters=[], job_id=167559915304449, job_parameters=[], number_in_job=620884104255456, original_attempt_run_id=620884104255456, overriding_parameters=None, repair_history=[], run_duration=None, run_id=620884104255456, run_name='Databricks SDK Job', run_page_url='https://adb-3587007519857700.0.azuredatabricks.net/?o=3587007519857700#job/167559915304449/run/620884104255456', run_type=<RunType.JOB_RUN: 'JOB_RUN'>, schedule=None, setup_duration=0, start_time=1731015142815, state=RunState(life_cycle_state=<RunLifeCycleState.TERMINATED: 'TERMINATED'>, queue_reason=None, result_state=<RunResultState.SUCCESS: 'SUCCESS'>, state_message='', user_cancelled_or_timedout=False), tasks=[RunTask(attempt_number=0, cleanup_duration=0, cluster_instance=None, condition_task=None, dbt_task=None,

Delete job

In [0]:
w.jobs.delete(job_id=j.job_id)

In [0]:
display(spark.sql("SELECT * FROM main.`dlt-sql`.dlt_sql_bronze limit 10"))

Year,First_Name,County,Sex,Count
2021,OLIVIA,Albany,F,16
2021,AMELIA,Albany,F,16
2021,SOPHIA,Albany,F,13
2021,AVA,Albany,F,13
2021,EMMA,Albany,F,12
2021,PENELOPE,Albany,F,11
2021,MADISON,Albany,F,10
2021,ABIGAIL,Albany,F,10
2021,HARPER,Albany,F,10
2021,ISLA,Albany,F,9


In [0]:
display(spark.sql("SELECT * FROM main.`dlt-sql`.dlt_sql_silver limit 10"))

Year_Of_Birth,First_Name,Count
2021,OLIVIA,16
2021,AMELIA,16
2021,SOPHIA,13
2021,AVA,13
2021,EMMA,12
2021,PENELOPE,11
2021,MADISON,10
2021,ABIGAIL,10
2021,HARPER,10
2021,ISLA,9


In [0]:
display(spark.sql("SELECT * FROM main.`dlt-sql`.dlt_sql_gold limit 10"))

First_Name,Total_Count
LIAM,1359
NOAH,1255
OLIVIA,924
EMMA,924
LUCAS,774
SOPHIA,754
JOSEPH,749
MIA,720
JACOB,711
AMELIA,695
